<a href="https://colab.research.google.com/github/bannis14/Portfolio-Projects/blob/main/Time_Series_Forecast/Linkedin_Silverkite_Parameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LinkedIn Silverkite - Parameter Tuning

## Libraries and Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# change directory
%cd /content/drive/MyDrive/Time Series Forecasting Product

/content/drive/MyDrive/Time Series Forecasting Product


In [ ]:
# Install Greykite
!pip install greykite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.5/22.5 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.9/499.9 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 84.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 80.2 MB/s eta 0:00:00
  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=731999 sha256=896414257ddce4899d7e5a382716f497fb7e805d7bbc4494fe27c224fda13ebf
  Stored in directory: /root/.cache/pip/wheels/d6/67/78/aa2e8d108639dd23a5e9e72a

In [ ]:
# Libraries
import numpy as np
import pandas as pd
from greykite.framework.templates.autogen.forecast_config import *
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.common.features.timeseries_features import *
from greykite.common.evaluation import EvaluationMetricEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results
from plotly.offline import iplot

In [ ]:
# Load the data
# YYYY-MM-DD
df = pd.read_csv('nyc_data.csv')
future_df = pd.read_csv('future.csv')
future_df.head()

,Date,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
0,1/1/2021,NaN,0,0,0,5.00,154.221
1,1/2/2021,NaN,0,0,0,11.11,264.805
2,1/3/2021,NaN,0,0,0,3.89,115.499
3,1/4/2021,NaN,0,0,0,6.67,124.650
4,1/5/2021,NaN,0,0,0,5.56,77.968


In [ ]:
# Merging both
df = pd.concat([df, future_df])
df = df.reset_index(drop=True)
df.head()

,Date,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
0,1/1/2015,720.000885,0,0,0,3.68,41.305
1,1/2/2015,581.276773,0,0,0,4.73,131.574
2,1/3/2015,754.117039,0,0,0,7.23,162.700
3,1/4/2015,622.252774,0,0,0,10.96,160.281
4,1/5/2015,785.373319,0,0,0,6.92,51.077


In [ ]:
# Inspecting df
df.tail()

,Date,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
2218,1/27/2021,NaN,0,0,0,3.33,39.664
2219,1/28/2021,NaN,0,0,0,1.67,195.314
2220,1/29/2021,NaN,0,0,0,-2.78,235.894
2221,1/30/2021,NaN,0,0,0,1.11,152.752
2222,1/31/2021,NaN,0,0,0,4.44,158.620


In [ ]:
df = df.rename(columns={"Demand": "y"})

## Silverkite Model Prep

In [ ]:
# Specifying Time Series names
metadata = MetadataParam(time_col="Date",
                         value_col="y",
                         freq="D",
                         train_end_date=pd.to_datetime("2020-12-31"))
metadata

MetadataParam(anomaly_info=None, date_format=None, freq='D', time_col='Date', train_end_date=Timestamp('2020-12-31 00:00:00'), value_col='y')

In [ ]:
# Growth terms possibilities
growth = dict(growth_term=["linear", "quadratic", "sqrt"])
growth

{'growth_term': ['linear', 'quadratic', 'sqrt']}

In [ ]:
# Seasonalities
seasonality = dict(yearly_seasonality="auto",
                   quarterly_seasonality="auto",
                   monthly_seasonality="auto",
                   weekly_seasonality="auto",
                   daily_seasonality="auto")
seasonality

{'yearly_seasonality': 'auto',
 'quarterly_seasonality': 'auto',
 'monthly_seasonality': 'auto',
 'weekly_seasonality': 'auto',
 'daily_seasonality': 'auto'}

In [ ]:
# Checking which countries are available and their holidays
get_available_holiday_lookup_countries(["US"])
get_available_holidays_across_countries(countries=["US"],
                                        year_start=2015,
                                        year_end=2021)

['Christmas Day',
 'Christmas Day (Observed)',
 'Columbus Day',
 'Halloween',
 'Independence Day',
 'Independence Day (Observed)',
 'Juneteenth National Independence Day',
 'Juneteenth National Independence Day (Observed)',
 'Labor Day',
 'Martin Luther King Jr. Day',
 'Memorial Day',
 "New Year's Day",
 "New Year's Day (Observed)",
 'Thanksgiving',
 'Veterans Day',
 'Veterans Day (Observed)',
 "Washington's Birthday"]

In [ ]:
# Specifying events
events = dict(holidays_to_model_separately=["New Year's Day"],
              holiday_lookup_countries=["US"],
              holiday_pre_num_days=2,
              holiday_post_num_days=2,
              holiday_pre_post_num_dict={"New Year's Day": (3, 1)},
              daily_event_df_dict={"elections": pd.DataFrame({
                  "date": ["2016-11-08", "2020-11-03"],
                  "event_name": ["elections"] * 2
              })})
events

{'holidays_to_model_separately': ["New Year's Day"],
 'holiday_lookup_countries': ['US'],
 'holiday_pre_num_days': 2,
 'holiday_post_num_days': 2,
 'holiday_pre_post_num_dict': {"New Year's Day": (3, 1)},
 'daily_event_df_dict': {'elections':          date event_name
  0  2016-11-08  elections
  1  2020-11-03  elections}}

In [ ]:
# Changepoints -> reflects the changes in trend
changepoints = dict(changepoints_dict=dict(method="auto"))

In [ ]:
# Regressors
regressors = dict(regressor_cols=["Easter", "Temperature", "Marketing"])
regressors

{'regressor_cols': ['Easter', 'Temperature', 'Marketing']}

In [ ]:
# Lagged Regressors
lagged_regressors = dict(lagged_regressor_dict={"Temperature": "auto",
                                                 "Easter": "auto",
                                                 "Marketing": "auto"})

In [ ]:
# Autoregression -> dependent on the forecasting horizon
autoregression = dict(autoreg_dict="auto")

In [ ]:
# Fitting algorithms
custom = dict(fit_algorithm_dict=[dict(fit_algorithm="linear"),
                                   dict(fit_algorithm="ridge"),
                                   dict(fit_algorithm="rf"),
                                   dict(fit_algorithm="gradient_boosting")])
custom

{'fit_algorithm_dict': [{'fit_algorithm': 'linear'},
  {'fit_algorithm': 'ridge'},
  {'fit_algorithm': 'rf'},
  {'fit_algorithm': 'gradient_boosting'}]}

## Silverkite Model

In [ ]:
# Build the model
model_components = ModelComponentsParam(growth=growth,
                                        seasonality=seasonality,
                                        events=events,
                                        changepoints=changepoints,
                                        regressors=regressors,
                                        lagged_regressors=lagged_regressors,
                                        autoregression=autoregression,
                                        custom=custom)

In [ ]:
# Cross-validation
evaluation_period = EvaluationPeriodParam(cv_min_train_periods=df.shape[0] - 180 - 31,
                                          cv_expanding_window=True,
                                          cv_max_splits=50,
                                          cv_periods_between_splits=16)

In [ ]:
# Evaluation Metric
evaluation_metric = EvaluationMetricParam(
    cv_selection_metric=EvaluationMetricEnum.RootMeanSquaredError.name)

In [ ]:
# Configuration
config = ForecastConfig(model_template=ModelTemplateEnum.SILVERKITE.name,
                        forecast_horizon=31,
                        metadata_param=metadata,
                        model_components_param=model_components,
                        evaluation_period_param=evaluation_period,
                        evaluation_metric_param=evaluation_metric)

In [ ]:
# Forecasting
forecaster = Forecaster()
result = forecaster.run_forecast_config(df=df,
                                        config=config)

Fitting 8 folds for each of 12 candidates, totalling 96 fits


/usr/local/lib/python3.10/dist-packages/greykite/algo/forecast/silverkite/forecast_silverkite.py:3433: UserWarning:

The following Fourier series terms are removed due to collinearity:
['sin1_toq_quarterly', 'cos1_toq_quarterly', 'sin2_toq_quarterly', 'cos2_toq_quarterly', 'sin3_toq_quarterly', 'cos3_toq_quarterly', 'cos4_tow_weekly']

/usr/local/lib/python3.10/dist-packages/greykite/algo/forecast/silverkite/forecast_silverkite.py:3433: UserWarning:

The following Fourier series terms are removed due to collinearity:
['sin1_toq_quarterly', 'cos1_toq_quarterly', 'sin2_toq_quarterly', 'cos2_toq_quarterly', 'sin3_toq_quarterly', 'cos3_toq_quarterly', 'cos4_tow_weekly']

/usr/local/lib/python3.10/dist-packages/greykite/algo/forecast/silverkite/forecast_silverkite.py:3433: UserWarning:

The following Fourier series terms are removed due to collinearity:
['sin1_toq_quarterly', 'cos1_toq_quarterly', 'sin2_toq_quarterly', 'cos2_toq_quarterly', 'sin3_toq_quarterly', 'cos3_toq_quarterly', 'cos4_

In [ ]:
# Visualization
fig = result.backtest.plot()
iplot(fig)

## Parameter Tuning results

In [ ]:
# CV results
cv_results = summarize_grid_search_results(
    grid_search=result.grid_search,
    decimals=1,
    score_func=EvaluationMetricEnum.RootMeanSquaredError.name)

In [ ]:
# Set the CV results index
cv_results["params"] = cv_results["params"].astype(str)
cv_results.set_index("params", drop=True, inplace=True)
cv_results

,rank_test_CORR,rank_test_R2,rank_test_MSE,rank_test_RMSE,rank_test_MAE,rank_test_MedAE,rank_test_MAPE,rank_test_MedAPE,rank_test_sMAPE,rank_test_Q80,...,std_test_OutsideTolerance5p,split0_train_OutsideTolerance5p,split1_train_OutsideTolerance5p,split2_train_OutsideTolerance5p,split3_train_OutsideTolerance5p,split4_train_OutsideTolerance5p,split5_train_OutsideTolerance5p,split6_train_OutsideTolerance5p,split7_train_OutsideTolerance5p,std_train_OutsideTolerance5p
params,,,,,,,,,,,,,,,,,,,,,
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",2,2,2,2,2,3,2,3,2,8,...,0.1,0.5,0.5,0.4,0.4,0.4,0.4,0.4,0.4,0.0
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",1,1,1,1,1,1,1,1,1,4,...,0.1,0.5,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.0
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",3,3,3,3,3,2,3,2,3,6,...,0.1,0.5,0.5,0.4,0.4,0.4,0.4,0.4,0.4,0.0
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",5,6,6,6,6,6,5,6,6,12,...,0.0,0.4,0.5,0.4,0.4,0.4,0.4,0.4,0.4,0.0
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",4,4,4,4,4,4,4,4,4,10,...,0.1,0.4,0.5,0.4,0.4,0.4,0.4,0.4,0.4,0.0
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",6,5,5,5,5,5,6,5,5,11,...,0.0,0.4,0.5,0.4,0.4,0.4,0.4,0.4,0.4,0.0
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'rf'})]",12,12,12,12,12,10,12,10,12,9,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.0
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'rf'})]",10,11,11,11,11,11,11,11,11,7,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.0
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'rf'})]",11,10,10,10,10,12,10,12,10,5,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.0


In [ ]:
# Looking at the best results
cv_results[["rank_test_RMSE", "mean_test_RMSE",
            "param_estimator__fit_algorithm_dict",
            "param_estimator__growth_term"]]

,rank_test_RMSE,mean_test_RMSE,param_estimator__fit_algorithm_dict,param_estimator__growth_term
params,,,,
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",2,47.8,{'fit_algorithm': 'linear'},linear
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",1,47.7,{'fit_algorithm': 'linear'},quadratic
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",3,47.9,{'fit_algorithm': 'linear'},sqrt
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",6,48.9,{'fit_algorithm': 'ridge'},linear
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",4,48.5,{'fit_algorithm': 'ridge'},quadratic
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",5,48.9,{'fit_algorithm': 'ridge'},sqrt
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'rf'})]",12,66.3,{'fit_algorithm': 'rf'},linear
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'rf'})]",11,64.6,{'fit_algorithm': 'rf'},quadratic
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'rf'})]",10,64.1,{'fit_algorithm': 'rf'},sqrt


In [ ]:
best_params = cv_results[cv_results.rank_test_RMSE == 1][["mean_test_RMSE",
                                                          "param_estimator__fit_algorithm_dict",
                                                          "param_estimator__growth_term"]].transpose()

In [ ]:
best_params.to_csv("Forecasting Product/best_params_silverkite.csv")